# Prevalência e Codon Usage

#### Adicionando os paths

In [1]:
import sys
sys.path.append('../')
sys.path.append('../Scripts/')

#### Importando bibliotecas

In [2]:
import collections
import json
from Global_Variables import *
import Aux_Functions as af
import numpy as np
import pandas as pd

#### Informações da população teste

In [3]:
genotype = "1b"

#### Acessando os arquivos das populações

In [4]:
Datasets_Reference = [("NS5A_1b_06.03.2018.aln","1b"),
                     ("NS5A_GEN1a_02.05.16","1a"),
                     ("NS5A_GEN3_05.05.16","3")]

In [5]:
"""
# Caso precise trabalhar com os dados de 1b

genotype_1b = af.fasta_to_dict("../Dados/" + "NS5A_1b_06.03.2018.aln")
genotype_1b_codons = {}
for sequence in genotype_1b.items():
    genotype_1b_codons[sequence[0]] = af.sequence_codons(sequence[1])
"""

'\n# Caso precise trabalhar com os dados de 1b\n\ngenotype_1b = af.fasta_to_dict("../Dados/" + "NS5A_1b_06.03.2018.aln")\ngenotype_1b_codons = {}\nfor sequence in genotype_1b.items():\n    genotype_1b_codons[sequence[0]] = af.sequence_codons(sequence[1])\n'

In [6]:
genotype_1a = af.fasta_to_dict("../Dados/" + "NS5A_1b_06.03.2018.aln")
genotype_1a_codons = {}
for sequence in genotype_1a.items():
    genotype_1a_codons[sequence[0]] = af.sequence_codons(sequence[1])

#### Contando a prevalência dos codons nas posições

In [7]:
df = pd.DataFrame(data=genotype_1a_codons)
df = df.transpose()
df_prevalence = df.apply(pd.value_counts)
df_prevalence = df_prevalence.apply(lambda x: x/len(df))

#### Adicionando codons que não apareceram em nenhuma posição

In [19]:
for codon in codontable.keys():
    if codon not in df_prevalence.index:
        df_prevalence.loc[codon] = pd.Series()

In [20]:
df_prevalence.head(10)

,24,28,30,31,32,54,58,92,93
AAA,NaN,NaN,0.002427,NaN,NaN,NaN,NaN,NaN,NaN
AAC,NaN,NaN,NaN,NaN,NaN,0.008495,NaN,NaN,NaN
AAG,0.009709,NaN,0.001214,NaN,NaN,NaN,NaN,NaN,NaN
AAT,NaN,NaN,NaN,NaN,NaN,0.038835,NaN,NaN,NaN
ACA,NaN,NaN,NaN,NaN,NaN,NaN,0.009709,0.004854,NaN
ACG,NaN,NaN,NaN,NaN,NaN,NaN,0.001214,0.008495,NaN
ACT,NaN,NaN,NaN,NaN,NaN,NaN,0.001214,NaN,NaN
AGG,NaN,NaN,0.004854,NaN,NaN,NaN,NaN,NaN,NaN
ATA,NaN,NaN,NaN,0.002427,NaN,NaN,NaN,NaN,NaN
ATG,NaN,0.01699,0.001214,0.033981,NaN,NaN,NaN,NaN,NaN


#### Criando a tabela da população

In [21]:
df_codons = pd.DataFrame(columns=["Genotype", "Position", "Codon", "Prevalence"])

In [22]:
for position in df_prevalence.columns:
    for codon in df_prevalence.index:
        new_row = [genotype, position, codon, df_prevalence.at[codon,position]]
        new_series = pd.Series(new_row, index = df_codons.columns)
        df_codons = df_codons.append(new_series, ignore_index=True)

In [23]:
df_codons = df_codons.fillna(0) # Not a Number values into 0

In [24]:
df_codons.head(10)

,Genotype,Position,Codon,Prevalence
0,1b,24,AAA,0.000000
1,1b,24,AAC,0.000000
2,1b,24,AAG,0.009709
3,1b,24,AAT,0.000000
4,1b,24,ACA,0.000000
5,1b,24,ACG,0.000000
6,1b,24,ACT,0.000000
7,1b,24,AGG,0.000000
8,1b,24,ATA,0.000000
9,1b,24,ATG,0.000000


#### Abrindo tabela de codon usage da população

In [25]:
df_codon_usage = pd.read_csv("../Dados/Codon_Usage_G1.csv", sep=";")

In [26]:
df_codon_usage.head(10)

,Codon,Codon_Usage
0,GCG,0.23
1,GCA,0.17
2,GCT,0.20
3,GCC,0.40
4,TGT,0.31
5,TGC,0.69
6,GAT,0.28
7,GAC,0.72
8,GAG,0.72
9,GAA,0.28


#### Adicionando as informações de codon usage a tabela de informações da população

In [27]:
df_join = pd.merge(df_codons, df_codon_usage, on=['Codon'])

In [28]:
df_join.head(10)

,Genotype,Position,Codon,Prevalence,Codon_Usage
0,1b,24,AAA,0.000000,0.32
1,1b,28,AAA,0.000000,0.32
2,1b,30,AAA,0.002427,0.32
3,1b,31,AAA,0.000000,0.32
4,1b,32,AAA,0.000000,0.32
5,1b,54,AAA,0.000000,0.32
6,1b,58,AAA,0.000000,0.32
7,1b,92,AAA,0.000000,0.32
8,1b,93,AAA,0.000000,0.32
9,1b,24,AAC,0.000000,0.71


#### Criando arquivo da tabela com informações da população e codon usage

In [29]:
df_join.to_csv("../Outputs/Prevalence_CodonUsage.csv", sep=";", index=False)